In [1]:
%config Completer.use_jedi=False
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

In [3]:
import base64
import requests
import torch
import os
import numpy as np
import argparse
from PIL import Image
import torchvision.transforms as transforms
from torch.autograd import Variable
import torchvision.utils as vutils
from network.Transformer import Transformer

from tqdm.notebook import tqdm
from test_from_code import transform

import onnx
import onnxruntime

In [4]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [5]:
styles = ["Hosoda", "Hayao", "Shinkai", "Paprika"]

models = {}

for style in tqdm(styles):
    model = Transformer()
    model.load_state_dict(torch.load(os.path.join("./pretrained_models", style + '_net_G_float.pth')))
    model.eval()
    models[style] = model

  0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
for style in tqdm(styles):
    model = models[style]
    x = torch.randn(1, 3, 224, 224, requires_grad=True)
    
    torch.onnx.export(model, 
                      x, 
                      f"../pretrained_models/onnx/{style}.onnx", 
                      export_params=True, 
                      opset_version=10, 
                      do_constant_folding=True, 
                      input_names=["input"], 
                      output_names=["output"], 
                      dynamic_axes={"input": 
                                    {0: "batch_size",
                                     2: "width", 
                                     3: "height"}
                                   }
                     )
    
    onnx_model = onnx.load(f"../pretrained_models/onnx/{style}.onnx")
    onnx.checker.check_model(onnx_model)
    
    ort_session = onnxruntime.InferenceSession(f"../pretrained_models/onnx/{style}.onnx")
    
    array = np.random.random((1, 3, 312, 450))
    array = np.array(array, dtype=np.float32)
    ort_inputs = {ort_session.get_inputs()[0].name: array}
    ort_outs = ort_session.run(None, ort_inputs)
    
    print(f"model {style} saved!")

  0%|          | 0/4 [00:00<?, ?it/s]

c:\Users\agarw\OneDrive\Documents\projects\AvatarCraft\cartoongan\notebooks\..\network\Transformer.py:172: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  (n - 1) / float(n)


FileNotFoundError: [Errno 2] No such file or directory: '../pretrained_models/onnx/Hosoda.onnx'

### Inference using onnx models

In [ ]:
onnx_model = onnx.load("../pretrained_models/onnx/Hayao.onnx")
onnx.checker.check_model(onnx_model)
ort_session = onnxruntime.InferenceSession(f"../pretrained_models/onnx/Hayao.onnx")

In [ ]:
%%time

array = np.random.random((1, 3, 300, 400))
array = np.array(array, dtype=np.float32)

ort_inputs = {ort_session.get_inputs()[0].name: array}
ort_outs = ort_session.run(None, ort_inputs)

NameError: name 'ort_session' is not defined